In [14]:
import pandas as pd
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from sklearn.model_selection import KFold

In [15]:
data=pd.read_csv('datafinal.csv')
data=data.drop(['t2mshan', 'sh850ct', 'sh700ct', 'u700ct', 'vo850ct', 'sktshan', 't2msib', 'u850ct', 'stct', 'rh700ct', 'sktsib', 'dz850sib', 'sstct', 'mslpsib', 'rh925ct', 'rh850ct', 'z850sib', 'sst', 'vo925ct', 'rh850', 'spsib', 'st', 'u925ct', 'u925', 'sh925ct', 'dmslpsib', 'dspsib', 'sh850', 'dspct', 'dz925sib', 'sh700', 'dmslpct', 'z925sib'],axis=1)
data.set_index('date', inplace = True)

In [16]:
train_size = int(0.8 * len(data))

y_train = data['cens'][:train_size]
X_train = data.drop('cens',axis=1)[:train_size]
y_test = data['cens'][train_size:]
X_test = data.drop('cens',axis=1)[train_size:]

target = data['cens']
feature = data.drop('cens',axis=1)

print("Ukuran data pelatihan yang telah diacak:", X_train.shape)

Ukuran data pelatihan yang telah diacak: (38185, 41)


Define HyperParameter Search and define optuna objective function

In [17]:
xgb_params = {
    'n_estimators': (100, 5000),
    'max_depth': (1, 10),
    'learning_rate': (0.0001, 0.1),
    'subsample': (0.5, 1),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1),
    'gamma': (0, 1),
    'min_child_weight': (1, 10)
}

catboost_params = {
    'n_estimators': (100, 5000),
    'max_depth': (1, 10),
    'learning_rate': (0.0001, 0.1),
    'subsample': (0.5, 1),
    'rsm': (0.5, 1),
    'reg_lambda': (0, 1),
    'random_strength': (1, 10)
}

lgb_params = {
    'n_estimators': (100, 5000),
    'max_depth': (1, 10),
    'learning_rate': (0.0001, 0.1),
    'subsample': (0.5, 1),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1),
    'colsample_bytree': (0.5, 1),
    'min_child_samples': (1, 20)
}


def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', *xgb_params['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *xgb_params['max_depth']),
        'learning_rate': trial.suggest_float('learning_rate', *xgb_params['learning_rate']),
        'subsample': trial.suggest_float('subsample', *xgb_params['subsample']),
        'reg_alpha': trial.suggest_float('reg_alpha', *xgb_params['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *xgb_params['reg_lambda']),
        'gamma': trial.suggest_float('gamma', *xgb_params['gamma']),
        'min_child_weight': trial.suggest_int('min_child_weight', *xgb_params['min_child_weight'])
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    y_pred = model.predict(X_test)
    mse = r2_score(y_test, y_pred)
    return mse

def objective_catboost(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', *catboost_params['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *catboost_params['max_depth']),
        'learning_rate': trial.suggest_float('learning_rate', *catboost_params['learning_rate']),
        'subsample': trial.suggest_float('subsample', *catboost_params['subsample']),
        'rsm': trial.suggest_float('rsm', *catboost_params['rsm']),
        'reg_lambda': trial.suggest_float('reg_lambda', *catboost_params['reg_lambda']),
        'random_strength': trial.suggest_int('random_strength', *catboost_params['random_strength'])
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    y_pred = model.predict(X_test)
    mse = r2_score(y_test, y_pred)
    return mse

def objective_lgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', *lgb_params['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *lgb_params['max_depth']),
        'learning_rate': trial.suggest_float('learning_rate', *lgb_params['learning_rate']),
        'subsample': trial.suggest_float('subsample', *lgb_params['subsample']),
        'reg_alpha': trial.suggest_float('reg_alpha', *lgb_params['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *lgb_params['reg_lambda']),
        'colsample_bytree': trial.suggest_float('colsample_bytree', *lgb_params['colsample_bytree']),
        'min_child_samples': trial.suggest_int('min_child_samples', *lgb_params['min_child_samples'])
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    y_pred = model.predict(X_test)
    mse = r2_score(y_test, y_pred)
    return mse

def optimize_model(objective, n_trials):
    study = optuna.create_study(direction='maximize')  # Menggunakan 'minimize' untuk mencari MSE terkecil
    study.optimize(objective, n_trials=n_trials)
    best_params = study.best_params
    return best_params

best_lgb_params = optimize_model(objective_lgb, n_trials=50)
print('LightGBM - Best Hyperparameters:', best_lgb_params)

best_catboost_params = optimize_model(objective_catboost, n_trials=50)
print('CatBoost - Best Hyperparameters:', best_catboost_params)

best_xgb_params = optimize_model(objective_xgb, n_trials=50)
print('XGBoost - Best Hyperparameters:', best_xgb_params)


[I 2023-07-29 00:05:13,011] A new study created in memory with name: no-name-59978b39-5afa-40d0-9116-46b99d983610
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-07-29 00:05:36,572] Trial 0 finished with value: 0.796433863360532 and parameters: {'n_estimators': 1345, 'max_depth': 10, 'learning_rate': 0.0467324007884916, 'subsample': 0.9068239285519033, 'reg_alpha': 0.8358103729425416, 'reg_lambda': 0.975093971737965, 'colsample_bytree': 0.727271935933674, 'min_child_samples': 12}. Best is trial 0 with value: 0.796433863360532.
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pa

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-07-29 00:10:19,247] Trial 11 finished with value: 0.7935747492843803 and parameters: {'n_estimators': 2197, 'max_depth': 1, 'learning_rate': 0.09975652834964593, 'subsample': 0.6346003503246039, 'reg_alpha': 0.6746451575120375, 'reg_lambda': 0.21491773624886656, 'colsample_bytree': 0.9922994597964369, 'min_child_samples': 17}. Best is trial 7 with value: 0.8067059414740994.
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is depre

[I 2023-07-29 00:12:49,126] Trial 22 finished with value: 0.8017754074620296 and parameters: {'n_estimators': 2404, 'max_depth': 4, 'learning_rate': 0.06213830596977289, 'subsample': 0.687923896923231, 'reg_alpha': 0.5495292971431415, 'reg_lambda': 0.9075264351324628, 'colsample_bytree': 0.5003800679497191, 'min_child_samples': 20}. Best is trial 16 with value: 0.8082354163850255.
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-07-29 00:13:05,405] Trial 23 finished with value: 0.8006594865142975 and parameters: {'n_estimators': 4250, 'max_depth': 2, 'learning_rate': 0.07460153064799481, 'subsample': 0.5778964419927335, 'reg_alpha': 0.7293234340668736, 'reg_lambda': 0.8777745795410683, 'colsamp

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-07-29 00:15:44,027] Trial 34 finished with value: 0.7909170503827265 and parameters: {'n_estimators': 589, 'max_depth': 2, 'learning_rate': 0.05668593097302552, 'subsample': 0.6437558773320577, 'reg_alpha': 0.4907417932845489, 'reg_lambda': 0.8533724360327108, 'colsample_bytree': 0.6035885940774733, 'min_child_samples': 13}. Best is trial 16 with value: 0.8082354163850255.
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprec

[I 2023-07-29 00:18:10,892] Trial 45 finished with value: 0.8021719979074163 and parameters: {'n_estimators': 1656, 'max_depth': 4, 'learning_rate': 0.0599878964659873, 'subsample': 0.8266384150605808, 'reg_alpha': 0.6070547098569186, 'reg_lambda': 0.8952713198601993, 'colsample_bytree': 0.6481452396477221, 'min_child_samples': 14}. Best is trial 16 with value: 0.8082354163850255.
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-07-29 00:18:21,701] Trial 46 finished with value: 0.8064980166883251 and parameters: {'n_estimators': 2070, 'max_depth': 2, 'learning_rate': 0.07256187027444631, 'subsample': 0.7616761307388931, 'reg_alpha': 0.6253262226109154, 'reg_lambda': 0.8467858747072465, 'colsamp

...

[I 2023-07-29 01:24:58,898] Trial 48 finished with value: 0.8076737660040602 and parameters: {'n_estimators': 3373, 'max_depth': 6, 'learning_rate': 0.017583024393364684, 'subsample': 0.6093604736758588, 'rsm': 0.8201897680024379, 'reg_lambda': 0.5592624665180438, 'random_strength': 5}. Best is trial 39 with value: 0.8093989426275512.
[I 2023-07-29 01:25:33,013] Trial 49 finished with value: 0.7826012390982756 and parameters: {'n_estimators': 4407, 'max_depth': 1, 'learning_rate': 0.029965187444945196, 'subsample': 0.5576833342579371, 'rsm': 0.8699258690605345, 'reg_lambda': 0.717446575120248, 'random_strength': 3}. Best is trial 39 with value: 0.8093989426275512.
[I 2023-07-29 01:25:33,013] A new study created in memory with name: no-name-21c69c5e-6616-41d3-8db2-1724288d9214


CatBoost - Best Hyperparameters: {'n_estimators': 4901, 'max_depth': 4, 'learning_rate': 0.018812193862372664, 'subsample': 0.5701092869059072, 'rsm': 0.7857242286786702, 'reg_lambda': 0.773020178734695, 'random_strength': 3}


[I 2023-07-29 01:37:24,814] Trial 0 finished with value: 0.8002251402564684 and parameters: {'n_estimators': 4299, 'max_depth': 4, 'learning_rate': 0.04317902306492365, 'subsample': 0.6648683873318781, 'reg_alpha': 0.2164527111382445, 'reg_lambda': 0.5239512937736286, 'gamma': 0.44386097124128576, 'min_child_weight': 8}. Best is trial 0 with value: 0.8002251402564684.
[I 2023-07-29 01:56:16,323] Trial 1 finished with value: 0.7946823087742512 and parameters: {'n_estimators': 4826, 'max_depth': 6, 'learning_rate': 0.0965733048024456, 'subsample': 0.594378118405551, 'reg_alpha': 0.4769904690017993, 'reg_lambda': 0.355800610652114, 'gamma': 0.7850590364439346, 'min_child_weight': 1}. Best is trial 0 with value: 0.8002251402564684.
[I 2023-07-29 02:06:21,383] Trial 2 finished with value: 0.7970455763511575 and parameters: {'n_estimators': 4029, 'max_depth': 3, 'learning_rate': 0.00759635795398872, 'subsample': 0.8902946132548588, 'reg_alpha': 0.2325148031313874, 'reg_lambda': 0.27740765498

[W 2023-07-29 05:00:58,631] Trial 19 failed with value None.


KeyboardInterrupt: 